In [5]:
from pathlib import Path
import pandas as pd
import pickle
import xlwings as xw
import numpy as np
IDX = pd.IndexSlice
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
file = Path("C:/Code/balplot/enerbal/src/files/stats/processing/gefahrene_kilometer_private_pkw_2000_2018/fahrleistungen_privater_pkw.xlsx")
sheet_names = [
    "Österreich",
    "Burgenland",
    "Kärnten", 
    "Niederösterreich",
    "Oberösterreich", 
    "Salzburg", 
    "Steiermark", 
    "Tirol", 
    "Vorarlberg", 
    "Wien"
]
dfs = {}

for sheet_name in sheet_names:
    dfs[sheet_name] = pd.read_excel(file, sheet_name=sheet_name)

In [7]:
index = [1999]
index.extend(list(range(2003,2015, 2)))
print(index)
df_output = pd.DataFrame(index=index, columns=sheet_names)

for enum,sheet_name in enumerate(sheet_names):
    
    df = dfs[sheet_name].iloc[:,:4]
    df.set_index(df.columns[0], inplace=True)
    # df.reset_index(inplace=True, drop=True)
    df = df[df.index.notnull()]
    
    # Get all indices with "Q-Statistik ..." as the row value
    i = []
    for idx in df[df.index.str.startswith("Q")].index:
        i.append(df.index.get_loc(idx))

    # Iter over indices and check row above for keyword "Zusammen"
    p=[]
    for idx in i:

        # if idx == i[-1] or idx == i[-2]:
        #     col_idx = 4
        #     row_idx_offset = 5
        # else:
        col_idx = 3
        row_idx_offset = 4

        g = df.iloc[idx-row_idx_offset:idx, -1]
        g.name = sheet_name
        if g.index[-1] == "Zusammen":
            # print("Hi")
            p.append(g)
        else:
            raise Error

        # print(p)

        # for e, v in enumerate(p):
        #     h = (e+1)*2-1
        #     try:
        #         df_output.iloc[h,enum] = v / 2            
        #         df_output.iloc[h-1,enum] = v / 2
        #     except:
        #         pass

[1999, 2003, 2005, 2007, 2009, 2011, 2013]


In [8]:
wb = xw.Book()
for sheet_name in sheet_names:
    try:
        wb.sheets.add(sheet_name)
    except:
        pass
    last_column = "A3"

    for v, idx in zip(p, index):
        print(v)
        # if idx == 1999:
            # ws.range('A3').value = v.to_frame().T.values
        ws = wb.sheets[sheet_name]
        ws.range(last_column).api.Font.Bold = True
        # ws.range(last_column[0] + '1').value = v
        ws.range(last_column).value = "Liter"
        # v = v.reset_index()
        # v = v.iloc[:,0]
        try:
            v.drop("Sonstiger", inplace=True)
        except:
            pass

        try:
            v.drop("Insgesamt", inplace=True)
        except:
            pass
        v.iloc[:2] *= np.array([8.4, 9.8])
        v["Zusammen"] = v.iloc[:2].sum(axis=0) 
        v.index.name = idx
        ws.range(last_column).value =  v / 1e9 # kWh -> TWh
        last_column = ws.range(last_column).end('right').offset(0,2).get_address(0,0)
        ws.range(last_column).end('up').offset(0,1).value = "TWh"

        print(last_column)


Energieeinsatz der Haushalte (Mikrozensus 1999/2000) - Fahrleistungen und Treibstoffeinsatz privater Pkw\nErgebnisse für Wien
Insgesamt          NaN
Benzin       407301777
Diesel       195340469
Zusammen     602642246
Name: Wien, dtype: object
D3
Energieeinsatz der Haushalte (Mikrozensus 1999/2000) - Fahrleistungen und Treibstoffeinsatz privater Pkw\nErgebnisse für Wien
Benzin       324428714
Diesel       284311832
Sonstiger      2888481
Zusammen     611629027
Name: Wien, dtype: object
G3
Energieeinsatz der Haushalte (Mikrozensus 1999/2000) - Fahrleistungen und Treibstoffeinsatz privater Pkw\nErgebnisse für Wien
Benzin       280791480
Diesel       274245655
Sonstiger      4611649
Zusammen     559648784
Name: Wien, dtype: object
J3
Energieeinsatz der Haushalte (Mikrozensus 1999/2000) - Fahrleistungen und Treibstoffeinsatz privater Pkw\nErgebnisse für Wien
Benzin       247789308
Diesel       275246211
Sonstiger      4596916
Zusammen     527632435
Name: Wien, dtype: object
M3
Energieeinsa